In [ ]:
# Imports

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import PIL.Image
from PIL import ImageOps
import PIL
import pathlib
import matplotlib.pyplot as plt
import datetime
import tensorboard
import IPython
import sklearn
import cv2
import subprocess
import sys

In [ ]:
# Dataset loading

data_path = pathlib.Path('/home/samer/Documents/Programming/AI50xIraq/Cancerdetection/archivecopy/Training/')
data_path_test = pathlib.Path('/home/samer/Documents/Programming/AI50xIraq/Cancerdetection/archivecopy/Testing/')

dataset_path = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='training',
    seed= 1,
    batch_size=5,
    image_size=(180, 180),
    color_mode="grayscale",
    shuffle=True)

dataset_path_val = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='validation',
    seed= 2,
    batch_size=5,
    image_size=(180, 180),
    color_mode="grayscale",
    shuffle=True)

dataset_path_test = tf.keras.utils.image_dataset_from_directory(
    data_path_test,
    labels= 'inferred',
    seed= 3,
    batch_size=5,
    image_size=(180, 180),
    color_mode="grayscale",
    shuffle=True)


In [ ]:
# catch and prefetch
AUTOTUNE=tf.data.AUTOTUNE
dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Counts and visualisations could be replaced by tensorboard mostly

class_names = dataset_path.class_names
print(class_names)

data_path = pathlib.Path('/home/samer/Documents/Programming/AI50xIraq/Cancerdetection/archivecopy/Training')
data_dir = pathlib.Path(data_path)
image_count = len(list(data_path.glob('*/*.jpg')))
print(image_count)

plt.figure(figsize=(1, 2))
for images, labels in dataset_path.take(1):
   for i in range(2):
       ax = plt.subplot(1, 2, i+1)
       ax.imshow(images[i].numpy().astype("uint8"))
       plt.title(class_names[labels[i]])
       plt.axis("off")
   plt.show()
        print("hello")
x= 0
for image_batch, labels_batch in dataset_path:
   print(image_batch.shape)
   print(labels_batch.shape)
   x = x+1
   continue
print(x)

image, label= next(iter(dataset_path.take(1)))
_ = plt.imshow(image.numpy().astype("uint8"))
_ = plt.title(get_label_name(label))



In [ ]:
# Model

num_classes = 2

model = tf.keras.Sequential([
    tf.keras.layers.Resizing(60, 60),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, 3, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001)),  
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),                                                                         #DP1
    tf.keras.layers.Conv2D(32, 3, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),                                                                         #DP2
    tf.keras.layers.Conv2D(64, 3, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),                                                                         #DP3
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),                                                                         #DP4
    tf.keras.layers.Dense(128, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),                                                                         #DP5
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
#########################################
result = model(image)
_ = plt.imshow(result)
plt.show()
                                                          #More visualisations
plt.figure(figsize=(1, 1))
for images, labels in dataset_path.take(1):
   result = model(images[1])
   print(result.numpy().min(), result.numpy().max())
   _ = plt.imshow(result)
   plt.show()
#########################################


model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits= True),
    metrics=['accuracy','mse'])

#TensorBoard
log_dir= "/home/samer/Documents/Programming/AI50xIraq/Cancerdetection/TBLog/" + datetime.datetime.now().strftime("%Y%M%D-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq=1)

#Save
checkpoint_path = "/home/samer/Documents/Programming/AI50xIraq/Cancerdetection/Checkpoints/cp-{epoch:04d}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                 save_weights_only=True,
                                                 #save_best_only=True, 
                                                 verbose=1,
                                                 #save_freq=10
                                                  )

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(
    dataset_path,
    epochs=15,                                    
    validation_data = dataset_path_val
    callbacks =[tensorboard_callback,
                 cp_callback
                 ])

model.evaluate(dataset_path_test, batch_size=5, verbose=2)

model.summary()


saving_path = pathlib.Path('/home/samer/Documents/Programming/AI50xIraq/Cancerdetection/SavedModel/')

tf.keras.models.save_model(model,
                           saving_path,
                           overwrite=True,
                           save_format='tf'
                           )

#To load the model use:
#loaded_model = tf.keras.models.load_model('/home/samer/Documents/Programming/AI50xIraq/Cancerdetection/SavedModel/')


In [ ]:
# Tensorboard

# for clearing use: rm -rf /home/samer/Documents/Programming/AI50xIraq/Cancerdetection/TBLog/


# Todo
- use the dataset to train, validate and test.               (done)
- use dropout to avoid overfitting.                          (done)
- save a copy to load when deploying.                        (done)
- deploy with tensorflow.js. or TFX                          (predictionful)
- add some augmentation to expand the dataset.               (done)
- add more classes to work on the full dataset.              (done)

# Record
- ELU + sigmoid around 98 val 97
- ELU + softmax around 97 val 98 dips though but best yet